In [18]:
from pynq import Overlay, allocate
import cv2
import numpy as np
import os

overlay = Overlay("/home/xilinx/pynq/overlays/binarizer/binarizer.bit")
dma = overlay.axi_dma_0

In [19]:
img = cv2.imread('/home/xilinx/images/input.png')
print('image loaded', img.shape)

image loaded (704, 1472, 3)


In [20]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
h, w = img_gray.shape

In [21]:
flat_img = img_gray.flatten().astype(np.uint8)

In [22]:
pad = 0
if len(flat_img) % 4 != 0:
    pad = 4 - (len(flat_img) % 4)
    flat_img = np.pad(flat_img, (0, pad), constant_values=0)
    print("Padded by:", pad, "pixels")

n_pixels = len(flat_img)
print('transfer size', n_pixels)

transfer size 1036288


In [23]:
input_buffer = allocate(shape=(n_pixels,), dtype=np.uint8)
output_buffer = allocate(shape=(n_pixels,), dtype=np.uint8)

input_buffer[:] = flat_img

In [24]:
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.transfer(input_buffer)

dma.recvchannel.wait()
dma.sendchannel.wait()

In [25]:
if pad > 0:
    out_arr = np.array(output_buffer[:-pad])
else:
    out_arr = np.array(output_buffer)

out_img = out_arr.reshape((h, w))

cv2.imwrite('/home/xilinx/images/output.jpg', out_img)

True

In [30]:
from pynq.overlays.base import BaseOverlay
base_overlay = BaseOverlay("base.bit", download=True)

In [31]:
from pynq.lib.video import *
hdmi_out = base_overlay.video.hdmi_out
hdmi_out.configure(VideoMode(1280, 720, 24))
hdmi_out.start()

In [32]:
frame = hdmi_out.newframe()
img = cv2.cvtColor(out_img, cv2.COLOR_GRAY2RGB)
frame[:] = cv2.resize(img, (1280, 720))
hdmi_out.writeframe(frame)

In [29]:
hdmi_out.stop()
del hdmi_out